# (TITLE OF THE PROJECT)
#### AIM - 
###### (LINKS)

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Date and time
from datetime import datetime

# Preprocessing and modeling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Utilities
import warnings
warnings.filterwarnings("ignore")

In [2]:
# LOAD DATASETS
domain = pd.read_csv("data/domain_properties.csv")
suburb = pd.read_csv("data/syd_sub_rev.csv")

In [3]:
# CHECKING DATA
print("Domain Properties\n")
print(domain.info(), "\n\n", domain.head, "\n\n", domain.describe)
print("\n\nSydney Suburb Review\n")
print(suburb.info(), "\n\n", suburb.head, "\n\n", suburb.describe)

Domain Properties

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     11160 non-null  int64  
 1   date_sold                 11160 non-null  object 
 2   suburb                    11160 non-null  object 
 3   num_bath                  11160 non-null  int64  
 4   num_bed                   11160 non-null  int64  
 5   num_parking               11160 non-null  int64  
 6   property_size             11160 non-null  int64  
 7   type                      11160 non-null  object 
 8   suburb_population         11160 non-null  int64  
 9   suburb_median_income      11160 non-null  int64  
 10  suburb_sqkm               11160 non-null  float64
 11  suburb_lat                11160 non-null  float64
 12  suburb_lng                11160 non-null  float64
 13  suburb_elevation          11160 non-null  

In [4]:
# COLUMN STANDARDISATION
domain.columns = domain.columns.str.strip().str.lower().str.replace(" ", "_")
suburb.columns = suburb.columns.str.strip().str.lower().str.replace(" ", "_")

In [5]:
# CLEANING SUBURB FIRST
# RENAMING COLUMNS FOR LATER MERGING
suburb.rename(columns={"name" : "suburb"}, inplace=True)

# CHECK FOR DUPLICATE
print(suburb["suburb"].nunique(), "unique suburbs out of", suburb.shape[0], "rows")
print(suburb["suburb"].duplicated().sum(), "duplicate suburb entries")

421 unique suburbs out of 421 rows
0 duplicate suburb entries


In [6]:
# REMOVING SYMBOLS AND CONVERTING TYPES
def clean_currency(val):
    if isinstance(val, str):
        return pd.to_numeric(val.replace("$", "").replace(",", ""), errors="coerce")
    return val

def clean_percent(val):
    if isinstance(val, str):
        return pd.to_numeric(val.replace("%", ""), errors="coerce")
    return val

def clean_int(val):
    if isinstance(val, str):
        return pd.to_numeric(val.replace(",", ""), errors="coerce")
    return val

suburb["population_(rounded)*"] = suburb["population_(rounded)*"].apply(clean_int)
suburb["median_house_price_(2020)"] = suburb["median_house_price_(2020)"].apply(clean_currency)
suburb["median_house_price_(2021)"] = suburb["median_house_price_(2021)"].apply(clean_currency)
suburb["median_house_rent_(per_week)"] = suburb["median_house_rent_(per_week)"].apply(clean_currency)
suburb["median_apartment_price_(2020)"] = suburb["median_apartment_price_(2020)"].apply(clean_currency)
suburb["median_apartment_rent_(per_week)"] = suburb["median_apartment_rent_(per_week)"].apply(clean_currency)
suburb["%_change"] = suburb["%_change"].apply(clean_percent)
suburb["public_housing_%"] = suburb["public_housing_%"].apply(clean_percent)

In [7]:
currency_cols = [
    "median_house_price_(2020)", "median_house_price_(2021)",
    "median_apartment_price_(2020)", "median_house_rent_(per_week)",
    "median_apartment_rent_(per_week)"
]

percent_cols = ["%_change", "public_housing_%"]
int_cols = ["population_(rounded)*"]

for col in currency_cols:
    suburb[col] = suburb[col].apply(clean_currency)

for col in percent_cols:
    suburb[col] = suburb[col].apply(clean_percent)

for col in int_cols:
    suburb[col] = suburb[col].apply(clean_int)

suburb.dtypes

suburb                                            object
region                                            object
population_(rounded)*                              int64
postcode                                           int64
ethnic_breakdown_2016                             object
median_house_price_(2020)                        float64
median_house_price_(2021)                        float64
%_change                                         float64
median_house_rent_(per_week)                     float64
median_apartment_price_(2020)                    float64
median_apartment_rent_(per_week)                 float64
public_housing_%                                 float64
avg._years_held                                  float64
time_to_cbd_(public_transport)_[town_hall_st]     object
time_to_cbd_(driving)_[town_hall_st]              object
nearest_train_station                             object
highlights/attractions                            object
ideal_for                      

In [8]:
# DROPPING COLUMNS
suburb.drop(columns=[
    "region",
    "ethnic_breakdown_2016",
    "nearest_train_station",
    "highlights/attractions",
    "ideal_for",
    "review_link"
], inplace=True, errors="ignore")

for col in ["time_to_cbd_(public_transport)_[town_hall_st]", "time_to_cbd_(driving)_[town_hall_st]"]:
    suburb[col] = suburb[col].str.extract(r"(\d+)").astype(float)

In [9]:
# CHECKING FOR NULLS IN SUBURB
suburb.isnull().sum().sort_values(ascending=False), suburb.dtypes

(median_apartment_price_(2020)                    122
 median_apartment_rent_(per_week)                  71
 avg._years_held                                   51
 median_house_rent_(per_week)                      19
 median_house_price_(2021)                         15
 time_to_cbd_(public_transport)_[town_hall_st]      4
 %_change                                           4
 median_house_price_(2020)                          3
 time_to_cbd_(driving)_[town_hall_st]               2
 traffic                                            2
 public_housing_%                                   2
 noise                                              1
 affordability_(buying)                             1
 nature                                             1
 public_transport                                   1
 affordability_(rental)                             1
 family-friendliness                                1
 pet_friendliness                                   1
 safety                     

In [10]:
# MAKING NULL NUMERIC VALS TO MEDIAN
median_cols = [
    'median_apartment_price_(2020)',
    'median_apartment_rent_(per_week)',
    'avg._years_held',
    'median_house_rent_(per_week)',
    'median_house_price_(2021)',
    'median_house_price_(2020)',
    '%_change',
    'traffic',
    'public_housing_%',
    'time_to_cbd_(public_transport)_[town_hall_st]',
    'time_to_cbd_(driving)_[town_hall_st]'
]

suburb[median_cols] = suburb[median_cols].fillna(suburb[median_cols].median())
    
suburb.drop(columns=["things_to_see/do","postcode"], inplace=True, errors="ignore")
suburb.sample(20)

,suburb,population_(rounded)*,median_house_price_(2020),median_house_price_(2021),%_change,median_house_rent_(per_week),median_apartment_price_(2020),median_apartment_rent_(per_week),public_housing_%,avg._years_held,...,traffic,public_transport,affordability_(rental),affordability_(buying),nature,noise,family-friendliness,pet_friendliness,safety,overall_rating
181,Avalon Beach,10000,1900000.0,2250000.0,18.42,850.0,790000.0,550.0,0.0,8.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
259,Greenwich,5750,2850000.0,3200000.0,12.28,1125.0,750000.0,560.0,1.0,13.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,Pyrmont,13500,1550000.0,2000000.0,29.03,800.0,1000000.0,730.0,5.0,13.7,...,3.0,6.0,4.0,3.0,7.0,4.0,7.0,6.0,6.0,5.5
208,Narrabeen,8250,2250000.0,2650000.0,17.78,1050.0,855000.0,580.0,1.0,9.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,Camperdown,11000,1350000.0,1800000.0,33.33,785.0,760000.0,620.0,4.0,11.7,...,3.0,5.0,4.0,3.0,7.0,4.0,6.0,7.0,5.0,5.2
314,Tregear,4000,460000.0,570000.0,23.91,330.0,725000.0,280.0,38.0,12.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,Haberfield,6750,2100000.0,2350000.0,11.90,800.0,725000.0,450.0,2.0,15.4,...,3.0,7.0,3.0,3.0,8.0,5.0,8.0,9.0,9.0,6.3
327,Miller,3500,570000.0,690000.0,21.05,400.0,725000.0,330.0,44.0,9.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,Dee Why,22000,1900000.0,2500000.0,31.58,825.0,775000.0,560.0,2.0,15.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,Spring Farm,4500,675000.0,780000.0,15.56,505.0,725000.0,370.0,0.0,4.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
cols = [
    "traffic", "public_transport", "affordability_(rental)", "affordability_(buying)",
    "nature", "noise", "family-friendliness", "pet_friendliness",
    "safety", "overall_rating"
]
zero_counts = {col: (suburb[col] == 0).sum() for col in cols}
print(zero_counts)

{'traffic': np.int64(303), 'public_transport': np.int64(302), 'affordability_(rental)': np.int64(302), 'affordability_(buying)': np.int64(302), 'nature': np.int64(302), 'noise': np.int64(302), 'family-friendliness': np.int64(302), 'pet_friendliness': np.int64(302), 'safety': np.int64(302), 'overall_rating': np.int64(302)}


In [12]:
suburb.drop(columns=cols, inplace=True, errors="ignore")
suburb

,suburb,population_(rounded)*,median_house_price_(2020),median_house_price_(2021),%_change,median_house_rent_(per_week),median_apartment_price_(2020),median_apartment_rent_(per_week),public_housing_%,avg._years_held,time_to_cbd_(public_transport)_[town_hall_st],time_to_cbd_(driving)_[town_hall_st]
0,Hornsby,23000,1150000.0,1400000.0,21.74,600.0,645000.0,460.0,3.0,12.9,40.0,35.0
1,Oatley,10500,1550000.0,1800000.0,16.13,670.0,780000.0,470.0,0.0,13.7,30.0,30.0
2,Dulwich Hill,14500,1500000.0,1900000.0,26.67,725.0,780000.0,475.0,2.0,14.0,20.0,20.0
3,Jannali,6250,1050000.0,1300000.0,23.81,610.0,620000.0,465.0,8.0,13.7,35.0,45.0
4,Waverton,3250,2650000.0,3400000.0,28.30,1000.0,1200000.0,630.0,1.0,12.0,10.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...
416,Kyeemagh,1000,1400000.0,1650000.0,17.86,637.5,725000.0,410.0,1.0,16.6,45.0,25.0
417,Northwood,1000,3400000.0,4200000.0,23.53,1500.0,725000.0,500.0,0.0,13.0,45.0,20.0
418,Watsons Bay,1000,4700000.0,5500000.0,17.02,637.5,1300000.0,1400.0,1.0,12.3,35.0,25.0
419,Lavender Bay,950,3400000.0,4200000.0,23.53,637.5,950000.0,590.0,3.0,13.0,20.0,15.0


In [13]:
# Verify no missing values remain
print(suburb.isnull().sum())

suburb                                           0
population_(rounded)*                            0
median_house_price_(2020)                        0
median_house_price_(2021)                        0
%_change                                         0
median_house_rent_(per_week)                     0
median_apartment_price_(2020)                    0
median_apartment_rent_(per_week)                 0
public_housing_%                                 0
avg._years_held                                  0
time_to_cbd_(public_transport)_[town_hall_st]    0
time_to_cbd_(driving)_[town_hall_st]             0
dtype: int64


In [14]:
# CLEANING DOMAIN DATASET
print("Domain dataset info\n", domain.info())
print("\n\nDomain dataset describe\n", domain.describe(include='all'))
print("\n\nDomain dataset sample rows\n", domain.sample(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     11160 non-null  int64  
 1   date_sold                 11160 non-null  object 
 2   suburb                    11160 non-null  object 
 3   num_bath                  11160 non-null  int64  
 4   num_bed                   11160 non-null  int64  
 5   num_parking               11160 non-null  int64  
 6   property_size             11160 non-null  int64  
 7   type                      11160 non-null  object 
 8   suburb_population         11160 non-null  int64  
 9   suburb_median_income      11160 non-null  int64  
 10  suburb_sqkm               11160 non-null  float64
 11  suburb_lat                11160 non-null  float64
 12  suburb_lng                11160 non-null  float64
 13  suburb_elevation          11160 non-null  int64  
 14  cash_r

In [17]:
# CONVERT DATE_SOLD TO PROPER FORMAT
domain["date_sold"] = pd.to_datetime(domain["date_sold"], format="%d/%m/%y", errors="coerce")

# CHECKING FOR ANY NULLS AFTER DATE CONVERSION
invalid_dates = domain[domain["date_sold"].isna()]
print(invalid_dates)

# CHECK FEATURES FOR OUTLIERS
domain[["num_bath", "num_bed", "num_parking", "property_size"]].describe()

Empty DataFrame
Columns: [price, date_sold, suburb, num_bath, num_bed, num_parking, property_size, type, suburb_population, suburb_median_income, suburb_sqkm, suburb_lat, suburb_lng, suburb_elevation, cash_rate, property_inflation_index, km_from_cbd]
Index: []


,num_bath,num_bed,num_parking,property_size
count,11160.000000,11160.000000,11160.000000,11160.000000
mean,2.073566,3.758961,2.017473,723.012366
std,1.184881,1.559743,1.454560,1048.983662
min,0.000000,0.000000,0.000000,7.000000
25%,1.000000,3.000000,1.000000,430.000000
50%,2.000000,4.000000,2.000000,600.000000
75%,3.000000,4.000000,2.000000,765.000000
max,46.000000,47.000000,50.000000,59100.000000


In [ ]:
print("Bath outliers:", domain[domain['num_bath'] > 10].shape[0])
print("Bed outliers:", domain[domain['num_bed'] > 10].shape[0])
print("Parking outliers:", domain[domain['num_parking'] > 10].shape[0])
print("Property size outliers:", domain[domain['property_size'] > 5000].shape[0])
domain.info

Bath outliers: 10
Bed outliers: 51
Parking outliers: 16
Property size outliers: 85
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   price                     11160 non-null  int64         
 1   date_sold                 11160 non-null  datetime64[ns]
 2   suburb                    11160 non-null  object        
 3   num_bath                  11160 non-null  int64         
 4   num_bed                   11160 non-null  int64         
 5   num_parking               11160 non-null  int64         
 6   property_size             11160 non-null  int64         
 7   type                      11160 non-null  object        
 8   suburb_population         11160 non-null  int64         
 9   suburb_median_income      11160 non-null  int64         
 10  suburb_sqkm               11160 non-null  float64       
 1